In [ ]:
import boto3
from langchain.embeddings import BedrockEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain_community.chat_models import BedrockChat
from langchain.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import ToolNode

In [26]:
# !pip install boto3
# !pip install langchain
# !pip install langchain-community
# !pip install pypdf
# !pip install faiss-cpu
# !pip install langgraph

In [10]:
bedrock_runtime = boto3.client( 
        service_name="bedrock-runtime",
        region_name="us-east-1",
    )

embeddings = BedrockEmbeddings(
        model_id="amazon.titan-embed-text-v1",
        client=bedrock_runtime,
        region_name="us-east-1",
    )

index_creator = VectorstoreIndexCreator(
        vectorstore_cls=FAISS,
        embedding=embeddings,
    )

llm = BedrockChat(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=bedrock_runtime,
    region_name="us-east-1",
    model_kwargs={"temperature": 0.0},
)

/tmp/ipykernel_5304/2122603636.py:17: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import ChatBedrock``.
  llm = BedrockChat(


In [29]:
llm.invoke("hy")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model_id': 'anthropic.claude-3-haiku-20240307-v1:0', 'usage': {'prompt_tokens': 9, 'completion_tokens': 12, 'total_tokens': 21}}, id='run-9c38ec86-fa21-42d6-a3c2-e1bd61291621-0')

In [18]:
loader = PyPDFLoader("Statement of Purpose Development Questionnaire.docx (1).pdf")
index_from_loader = index_creator.from_loaders([loader])
index_from_loader.vectorstore.save_local("/tmp")

Ignoring wrong pointing object 238 0 (offset 0)


In [20]:
faiss_index = FAISS.load_local("/tmp", embeddings, allow_dangerous_deserialization=True)
retriever=faiss_index.as_retriever()

In [27]:
retriever_tool=create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return the the documents related to question",
    )

tools=[retriever_tool]
retrieve=ToolNode([retriever_tool])

In [36]:
# llm_with_tools = llm.bind_tools(tools=tools)

In [40]:
from typing import Annotated, Literal, Sequence, TypedDict
from langgraph.graph import END, StateGraph, START
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain_aws import ChatBedrock

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=dict(temperature=0),
    region_name = "us-east-1"
)

llm_with_tools = llm.bind_tools(tools=tools)

In [39]:
# !pip install langchain_aws

In [43]:
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=dict(temperature=0),
    region_name = "us-east-1"
)

llm_with_tools = llm.bind_tools(tools=tools)


In [62]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# Sample question bank
question_bank = [
    "What is machine learning?",
    "Can you explain the difference between supervised and unsupervised learning?",
    "What is overfitting and how can you prevent it?"
]

# Agent state definition

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    question_index: int
    validated_answers: list
    current_question: str
    user_answer: str
    answer_validated: bool

# 1. Greet the user
def greet_user(state: AgentState) -> AgentState:
    print("🤖 Hi there! I'm your interview assistant. Let's get started.")
    return state

# 2. Wait for user's greeting
def wait_for_user_greeting(state: AgentState) -> AgentState:
    greeting = input("You: ").strip().lower()
    if any(word in greeting for word in ["hi", "hello", "hey", "good", "ready"]):
        return state
    else:
        print("🤖 Just say hi when you're ready.")
        return wait_for_user_greeting(state)

# 3. Ask the current question from the question bank
def ask_question(state: AgentState) -> AgentState:
    index = state.get("question_index", 0)
    question = question_bank[index]
    print(f"🤖 Question {index + 1}: {question}")
    state["current_question"] = question
    return state

# 4. Get the user's answer (CLI input)
def get_user_answer(state: AgentState) -> AgentState:
    state["user_answer"] = input("Your answer: ").strip()
    return state

# 5. Validate the answer using Claude and retriever
def validate_answer(state: AgentState, tools) -> AgentState:
    # Use the retriever tool to fetch relevant docs
    retriever_tool = tools[0]
    retrieved_docs = retriever_tool.invoke({"question": state["current_question"]})

    # Call Claude with the answer and the retrieved context
    context = "\n".join(doc.page_content for doc in retrieved_docs)
    prompt = f"""Here is a user's answer to a question.

    Question: {state["current_question"]}
    Answer: {state["user_answer"]}

    Use the following context to validate the answer:
    {context}

    Is the answer correct and complete? Reply 'yes' or 'no' and explain why.
    """

    response = llm.invoke([HumanMessage(content=prompt)])
    print("🤖 Validation result:", response.content)

    if "yes" in response.content.lower():
        state["answer_validated"] = True
    else:
        state["answer_validated"] = False
    return state

# 6. Ask follow-up question
def ask_follow_up(state: AgentState) -> AgentState:
    follow_up = f"Can you clarify or expand more on this: {state['current_question']}?"
    print(f"🤖 Follow-up: {follow_up}")
    return state

# 7. Save validated answer and advance to next question
def save_valid_answer(state: AgentState) -> AgentState:
    state["validated_answers"].append({
        "question": state["current_question"],
        "answer": state["user_answer"]
    })
    state["question_index"] += 1
    return state

def wait_for_answer(state: AgentState) -> AgentState:
    state["user_answer"] = input("Your answer: ").strip()
    return state


# 8. End the interview
def end_interview(state: AgentState) -> AgentState:
    print("🤖 That's all for now. Thanks for your answers!")
    print("📄 Summary of your validated answers:")
    for i, entry in enumerate(state["validated_answers"], 1):
        print(f"{i}. {entry['question']} ➡ {entry['answer']}")
    return state


In [ ]:
# from langgraph.graph import StateGraph, ToolNode, Node
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

# Create your graph
graph = StateGraph(AgentState)

# Add nodes
graph.add_node("start", greet_user)
graph.add_node("wait_for_greeting", wait_for_user_greeting)
graph.add_node("ask_question", ask_question)
graph.add_node("wait_for_answer", wait_for_answer)
# graph.add_node("validate_answer", ToolNode([retriever_tool]))
graph.add_node("validate_answer", validate_answer)
graph.add_node("follow_up", ask_follow_up)
graph.add_node("save_answer", save_valid_answer)
graph.add_node("end", end_interview)

# Define the flow
graph.set_entry_point("start")
graph.add_edge("start", "wait_for_greeting")
graph.add_edge("wait_for_greeting", "ask_question")
graph.add_edge("ask_question", "wait_for_answer")
graph.add_edge("wait_for_answer", "validate_answer")

# Decision logic after validation
def validation_router(state: AgentState) -> str:
    if state.get("answer_validated", False):
        return "save_answer"
    else:
        return "follow_up"

graph.add_conditional_edges("validate_answer", validation_router)

# Follow-up response path
graph.add_edge("follow_up", "wait_for_answer")

# Valid path: save then continue or finish
def next_step(state: AgentState) -> str:
    if state["question_index"] + 1 < len(question_bank):
        return "ask_question"
    else:
        return "end"

graph.add_conditional_edges("save_answer", next_step)

# Compile graph
interview_graph = graph.compile()


In [67]:
app=graph.compile()

In [69]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    print("issue")
    # This requires some extra dependencies and is optional
    pass

issue


In [ ]:
# Example initial state
initial_state = {
    "messages": [],
    "question_index": 0,
    "validated_answers": [],
    "current_question": "What's your name?",
    "user_answer": "",
    "answer_validated": False
}

# Invoke the interview graph correctly without passing state as a keyword
interview_graph.invoke(input=initial_state)  # Pass the state as input directly


🤖 Hi there! I'm your interview assistant. Let's get started.
